In [6]:
from PIL import Image, ImageEnhance
import os
import numpy as np
import math
from datetime import datetime

In [7]:
input_folder = "Input"
filename = 'test2.jpg'

with Image.open(os.path.join(input_folder, filename)) as img:
    img.load()

x_size, y_size = img.size                       # размеры изображения
im_arr = []                                     # массив пикселей
br_arr = []                                     # массив яркостей

pixels = list(img.getdata())

im_arr = [pixels[i * x_size:(i + 1) * x_size] for i in range(y_size)]


# temp_arr = []
for j in range(y_size):
    a = []
    # b = []
    for i in range(x_size):
        # br_arr[j][i] = int(sum(im_arr[j][i])/3)
        a.append(int(sum(im_arr[j][i])/3))
        # b.append([im_arr[j][i]])
    br_arr.append(a)

# br_arr = np.array(br_arr)
# print(br_arr)

for j in range(10):
    row = []
    for i in range(10):
        row.append((im_arr[j][i], br_arr[j][i]))
    # print(row)
# br_img = Image.fromarray(np.asarray(br_arr))
# br_img.show()


In [8]:
# двумерное преобразование Фурье
def  DFT_2D(array_2d):
    x_size = len(array_2d[0])
    y_size = len(array_2d)

    h = 1/(x_size*y_size)
    G = []
    pi = math.pi


    for w in range(y_size):
        row = []
        print(w)
        for u in range(x_size):
            a = 0
            b = 0
            for m in range(y_size):
                for n in range(x_size):
                    a += array_2d[m][n] * math.cos(-2*pi*(m*u/y_size+n*w/x_size))
                    b += array_2d[m][n] * math.sin(-2*pi*(m*u/y_size+n*w/x_size))
            c = complex(a, b)*h
            row.append(c)
        G.append(row)
        
    return G

def reDFT_2D(G):
    x_size = len(G[0])
    y_size = len(G)

    h = 1
    array_2d = []
    pi = math.pi

    for m in range(y_size):
        row = []
        print(m)
        for n in range(x_size):
            a = 0
            b = 0
            for w in range(y_size):
                for u in range(x_size):
                    a += G[w][u] * math.cos(2*pi*(m*u/y_size+n*w/x_size))
                    b += G[w][u] * math.sin(2*pi*(m*u/y_size+n*w/x_size))
            c = complex(a, b)*h
            row.append(c)
        array_2d.append(row)
    
    return array_2d

def clean_high_freqs(G, level):
    if (level > 1) or (level < 0):
        return None
    else:
        num_x = len(G[0])
        num_y = len(G)
        # core_x = math.floor(level*num_x)
        # core_y = math.floor(level*num_y)
        b_x = math.ceil((1-level)*num_x/2)
        b_y = math.ceil((1-level)*num_y/2)

        G_new = []
        for m in range(y_size):
            row = []
            for n in range(x_size):
                a = G[m][n]
                if b_x <= n <= (x_size - b_x):
                    if b_y <= m <= (y_size - b_y):
                        a = 0
                row.append(a)
            G_new.append(row)
        
        return G_new
    

In [9]:
dft = DFT_2D(br_arr)
print('dft')
dft = clean_high_freqs(dft, 0.8)
print('clean dft')
redft = reDFT_2D(dft)
print('redft')
print(br_arr[0])
print(redft[0])


0
1
2
3


KeyboardInterrupt: 

In [ ]:
test_img = []
for m in range(len(redft)):
    row = []
    for n in range(len(redft[0])):
        row.append(int(redft[m][n].real))
    test_img.append(row)

test_img = Image.fromarray(np.asarray(test_img))

now = datetime.now()

test_img.show()

current_time = now.strftime("%Y_%m_%d_%H_%M_%S")

test_img = test_img.save("Output/" + current_time + ".png")
